# Import / Config

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
import edurel.utils.dbcon as dbcu
import edurel.utils.db as dbu
import edurel.utils.duckdb as ddbu
import edurel.utils.llm as llmu
import edurel.utils.llmchat as llmc
import edurel.widgets.mermaid_viz as mmw
import edurel.widgets.chatman as cmw
import edurel.utils.misc as mu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


# Database

In [ ]:
# con.close()

In [ ]:
con = dbcu.ccfraud()
additional_fks = {}
# con = dbcu.company_en()
# additional_fks = {"OrgUnit": ["Employee|Head|EID", "OrgUnit|SuperUnit|OUID"]}

db = dbu.DbHandler(con,additional_fks=additional_fks)
schema = db.schema_yaml_str(["nullable", "fkname"])

In [ ]:
glm46 = llmu.stats_c(llmu.GLM46)
chat = llmc.LLMChat(glm46)

In [ ]:
viz = mmw.MermaidViz(chat._graph.get_graph().draw_mermaid(), height="300px", width="100%")
viz.display()

In [ ]:
system_prompt = """
You are an expert SQL query generator. 
Convert natural language questions into valid SQL queries.
use postgres syntax.
"""
chat.set_system_prompt(system_prompt)


In [ ]:
schema_prompt = f"""
The database schema is defined as follows:
{schema}
"""
# print(schema_prompt)
chat.add_user_message(schema_prompt)

In [ ]:
q1a = """
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
sorted by tyear;
"""

user_msg = f"""
{q1a}
Return only the SQL query. Do not explain.
"""
chat.add_user_message(user_msg)

In [ ]:
user_msg = f"""
column frac has an error in its calculation. please fix it.
"""
chat.add_user_message(user_msg)

# ChatMan

In [ ]:
model = llmu.stats_c(llmu.OPUS41)
chat = llmc.LLMChat(model)
chatman = cmw.ChatMan(chat)
chatman.display()

In [ ]:
"generate rewritten query"

In [ ]:
sql = mu.sql_extract(chat.get_message(-1).content)
print(sql)
db.sql_print(sql)

# Show Conversation

In [ ]:
print(chat.show_conversation())

# SQL Execution

In [ ]:
sql1 = mu.sql_extract(chat.get_message(-1).content)
print(sql1)
db.sql_print(sql1)

In [ ]:
sql2 = mu.sql_extract(chat.get_message(-1).content)
print(sql2)
db.sql_print(sql2)

In [ ]:
sql3 = mu.sql_extract(chat.get_message(-1).content)
print(sql3)
db.sql_print(sql3)